In [1]:
import pandas as pd
import sys
read_path = "~/Dropbox (ASU)/Google Analytics/all/"


In [2]:
# Set path and get helpers
sys.path.append("/Users/saaryalov/GoogleAnalyticsKaggle/")


In [3]:
%%time
train = pd.read_csv(read_path+"extracted_fields_train.gz", dtype={'date': str, 'fullVisitorId': str, 'sessionId':str}, nrows=None)
test = pd.read_csv(read_path+"extracted_fields_test.gz", dtype={'date': str, 'fullVisitorId': str, 'sessionId':str}, nrows=None)

CPU times: user 12.4 s, sys: 870 ms, total: 13.3 s
Wall time: 12.1 s


In [4]:
train.columns

Index(['channelGrouping', 'date', 'fullVisitorId', 'sessionId', 'visitId',
       'visitNumber', 'visitStartTime', 'device.browser',
       'device.deviceCategory', 'device.isMobile', 'device.operatingSystem',
       'geoNetwork.city', 'geoNetwork.continent', 'geoNetwork.country',
       'geoNetwork.metro', 'geoNetwork.networkDomain', 'geoNetwork.region',
       'geoNetwork.subContinent', 'totals.bounces', 'totals.hits',
       'totals.newVisits', 'totals.pageviews', 'totals.transactionRevenue',
       'trafficSource.adContent', 'trafficSource.campaign',
       'trafficSource.isTrueDirect', 'trafficSource.keyword',
       'trafficSource.medium', 'trafficSource.referralPath',
       'trafficSource.source'],
      dtype='object')

In [5]:
train["totals.transactionRevenue"].fillna(0,inplace=True)


In [6]:
# Change to DateTime
for df in [train, test]:
    df['date'] = pd.to_datetime(df['visitStartTime'], unit='s')
    df['sess_date_dow'] = df['date'].dt.dayofweek
    df['sess_date_hours'] = df['date'].dt.hour
    df['sess_date_dom'] = df['date'].dt.day
    
    del df['date']
    del df['visitStartTime']


In [7]:
print(f"Notice that {train.shape} is different then used on kernels, it is because in most kernels low variance features were dropped")

Notice that (903653, 31) is different then used on kernels, it is because in most kernels low variance features were dropped


# Device FE

In [8]:
from helpers import *
device_ls = ['device.browser',
       'device.deviceCategory', 'device.isMobile', 'device.operatingSystem']
for each in device_ls:
    result = sort_by_rev(train,each,kind="sum")
    sm = sum(result["totals.transactionRevenue"]>0)
    print(f"{each} has levels >0 in : {sm} with shape : {result.shape}")


device.browser has levels >0 in : 9 with shape : (54, 3)
device.deviceCategory has levels >0 in : 3 with shape : (3, 3)
device.isMobile has levels >0 in : 2 with shape : (2, 3)
device.operatingSystem has levels >0 in : 7 with shape : (20, 3)


## Operating System

In [9]:
# Manual FE
def manual_ohe(col,level,name=None):
    # The goal of this function is to manually OHE features 
    assert isinstance(col,str)
    if isinstance(level,str):
        level = level.replace(" ","_")
        train[col+"__"+level] = train[col].map(lambda x: (x == level)*1)
        test[col+"__"+level] = test[col].map(lambda x: (x == level)*1)
    elif isinstance(level,list):
        train[col+"__"+name] = train[col].map(lambda x: (x in level)*1)
        test[col+"__"+name] = train[col].map(lambda x: (x in level)*1)
    else:
        print("ERROR!")

In [10]:
computer = ["Windows","Macintosh","Linux","Chrome OS ",'Firefox OS',"FreeBSD","SunOS",'OpenBSD']
phone = ["Android","iOS","Windows Phone","Samsung",'BlackBerry','Nokia',"NTT DoCoMo"]
gaming = ['Xbox','Nintendo Wii','Nintendo WiiU','Nintendo 3DS']
other = ["(not set)"]

def os_mapping(row):
    # The goal of this function is to map values for a new bucketed OS feature
    if row in computer:
        row = "Computer"
    elif row in phone:
        row = "Phone"
    elif row in gaming:
        row = "Gaming"
    else:
        row = "Other"
    return row
train["device.operatingSystem__Bucket"] = train["device.operatingSystem"].map(os_mapping)
test["device.operatingSystem__Bucket"] = test["device.operatingSystem"].map(os_mapping)


   
# Manual FE

is_linux_based = ["Linux","FreeBSD","SunOS",'OpenBSD']
manual_ohe("device.operatingSystem",is_linux_based,"Linux")



to_ohe = ["Macintosh","iOS","Windows","Android","Chrome OS"]
for level in to_ohe:
    manual_ohe("device.operatingSystem",level)

# Other useful features?    
train["device.operatingSystem__Apple"] = train["device.operatingSystem__Macintosh"]+train["device.operatingSystem__iOS"]
train["device.operatingSystem__Google"] = train["device.operatingSystem__Android"]+train["device.operatingSystem__Chrome_OS"]

test["device.operatingSystem__Apple"] = train["device.operatingSystem__Macintosh"]+train["device.operatingSystem__iOS"]
test["device.operatingSystem__Google"] = train["device.operatingSystem__Android"]+train["device.operatingSystem__Chrome_OS"]


In [11]:
sort_by_rev(train,'device.browser',kind="sum").head(10)
# We can let categorical level selection handle this

,device.browser,Frequency,totals.transactionRevenue
0,Chrome,0.686507,1.383105e+12
1,Safari,0.201676,5.239748e+10
2,Firefox,0.041021,8.931632e+10
3,Internet Explorer,0.021441,8.255550e+09
4,Edge,0.011293,6.482970e+09
5,Android Webview,0.008704,1.148500e+08
6,Safari (in-app),0.007580,1.537700e+08
7,Opera Mini,0.006794,0.000000e+00
8,Opera,0.006245,2.153500e+08
9,UC Browser,0.002686,0.000000e+00


# geoNetwork

In [12]:
geo_list = ['geoNetwork.city', 'geoNetwork.continent', 'geoNetwork.country',
       'geoNetwork.metro', 'geoNetwork.networkDomain', 'geoNetwork.region',
       'geoNetwork.subContinent']
for each in geo_list:
    result = sort_by_rev(train,each,kind="sum")
    sm = sum(result["totals.transactionRevenue"]>0)
    print(f"{each} has levels >0 in : {sm} with shape : {result.shape} or {round(sm/result.shape[0],4)*100}%")

geoNetwork.city has levels >0 in : 134 with shape : (649, 3) or 20.65%
geoNetwork.continent has levels >0 in : 6 with shape : (6, 3) or 100.0%
geoNetwork.country has levels >0 in : 69 with shape : (222, 3) or 31.080000000000002%
geoNetwork.metro has levels >0 in : 46 with shape : (94, 3) or 48.94%
geoNetwork.networkDomain has levels >0 in : 732 with shape : (28064, 3) or 2.6100000000000003%
geoNetwork.region has levels >0 in : 79 with shape : (376, 3) or 21.01%
geoNetwork.subContinent has levels >0 in : 19 with shape : (23, 3) or 82.61%


## geoNetwork.city

We can also include external information such as income in city, number of big tech jobs in city, number of universities in city. Etc etc..

Taken from https://www.google.com/about/locations/?region=north-america&office=mountain-view


In [13]:
def geo_by_size_spenders(col):
    # By Size
    print("Working on size for ",col)
    sorted_size = sort_by_rev(train,col,kind="sum").loc[0:50,]
    top_10_size = sorted_size.loc[0:10,col].tolist()
    top_25_size = sorted_size.loc[0:25,col].tolist()
    top_50_size = sorted_size.loc[0:50,col].tolist()
    
    # By Spending
    print("Working on spending for ", col)
    sorted_rev = sorted_size.sort_values("totals.transactionRevenue",ascending=False)
    top_10_spenders = sorted_rev.loc[0:10,col].tolist()
    top_25_spenders = sorted_rev.loc[0:25,col].tolist()
    top_50_spenders = sorted_rev.loc[0:50,col].tolist()
    
    # Make Features
    print("Making Features size features for ", col)
    manual_ohe(col,top_10_size,"top_10_size")
    manual_ohe(col,top_25_size,"top_25_size")
    manual_ohe(col,top_50_size,"top_50_size")
    
    print("Making Features spending features for ", col)
    manual_ohe(col,top_10_spenders,"top_10_spenders")
    manual_ohe(col,top_25_spenders,"top_25_spenders")
    manual_ohe(col,top_50_spenders,"top_50_spenders")

In [14]:
# Make geo by size feature
geo_by_size_spenders("geoNetwork.city")

# Make Silicon Valey Feature
silicon_valey = ["Mountain View","San Francisco","Sunnyvale","San Jose","Palo Alto"]
manual_ohe("geoNetwork.city",silicon_valey,"silicon_valey")


# Has Google Office FE features
has_google_office_north_america = ["Ann Arbor","Atlanta","Austin",
                         "Boulder","Cambridge","Chapel Hill","Chicago","Irvine"
                        "Kirkland","Los Angeles","Kitchener","Miami","Montreal","Mountain View"
                                   "New York","Pittsburgh","Playa Vista","Reston","San Bruno","San Diego","San Francisco",
                                   "Seattle","Sunnyvale","Toronto","Washington DC"]
has_google_office_latin_america = ["Belo Horizonte","Bogota","Buenos Aires","Mexico City"
                                  "Santiago","Sao Paulo"]   
has_google_office_europe = ["Aarhus","Amsterdam","Athens","Berlin","Brussels","Copenhagen",
                           "Dublin","Hamburg","Lisbon","London","Madrid","Milan","Moscow","Munich","Oslo","Paris",
                           "Prague","Stockholm","Vienna","Warsaw","Wroclaw","Zurich"]
has_google_office_asia = ["Bangalore","Bangkok","Beijing","Guangzhou","Gurgaon","Hong Kong","Hyderabad",
                         "Jakarta","Kuala Lumpur","Melbourne","Mumbai","Seoul","Shanghai","Singapore",
                          "Sydney","Taipei","Tokyo"]
has_google_office_middle_east = ["Dubai","Haifa","Istanbul","Johannesburg","Tel Aviv"]

# Manual OHE       
manual_ohe("geoNetwork.city",has_google_office_north_america,"google_at_north_america")
manual_ohe("geoNetwork.city",has_google_office_latin_america,"google_at_latin_america")
manual_ohe("geoNetwork.city",has_google_office_europe,"google_at_europe")
manual_ohe("geoNetwork.city",has_google_office_asia,"google_at_asia")
manual_ohe("geoNetwork.city",has_google_office_middle_east,"google_at_middle_east")

# Sum of all
train["geoNetwork.city__has_google"] = train["geoNetwork.city__google_at_north_america"]+train["geoNetwork.city__google_at_latin_america"] + train["geoNetwork.city__google_at_europe"] +train["geoNetwork.city__google_at_asia"] + train["geoNetwork.city__google_at_middle_east"] 
test["geoNetwork.city__has_google"] = test["geoNetwork.city__google_at_north_america"] + test["geoNetwork.city__google_at_latin_america"]+ test["geoNetwork.city__google_at_europe"] +test["geoNetwork.city__google_at_asia"] + test["geoNetwork.city__google_at_middle_east"] 


Working on size for  geoNetwork.city
Working on spending for  geoNetwork.city
Making Features size features for  geoNetwork.city
Making Features spending features for  geoNetwork.city


## geoNetwork.country

In [15]:
# Make geo by size feature
geo_by_size_spenders("geoNetwork.country")


# Countries that have google
google_at_north_america = ["United States","Canada"]
google_at_latin_america = ["Brazil","Colombia","Argentina","Mexico","Chile"]
google_at_europe = ["Denmark","Netherlands","Greece","Germany",
                    "Belgium","Ireland","Portugal","United Kingdom",
                   "Spain","Italy","Russia","Norway","France","Czech Republic",
                    "Sweden","Austria","Poland","Switzerland"]
google_at_asia = ["India","Thailand","China","India","Hong Kong",
                 "Indonesia","Malaysia","Australia","South Korea","Singapore",
                 "Taiwan","Japan"]
google_at_middle_east = ["United Arab Emirates","Israel","Turkey","South Africa"]

# Manual OHE       
manual_ohe("geoNetwork.country",google_at_north_america,"google_at_north_america")
manual_ohe("geoNetwork.country",google_at_latin_america,"google_at_latin_america")
manual_ohe("geoNetwork.country",google_at_europe,"google_at_europe")
manual_ohe("geoNetwork.country",google_at_asia,"google_at_asia")
manual_ohe("geoNetwork.country",google_at_middle_east,"google_at_middle_east")

# Sum of all
train["geoNetwork.country__has_google"] = train["geoNetwork.country__google_at_north_america"]+ train["geoNetwork.country__google_at_latin_america"] + train["geoNetwork.country__google_at_europe"] +train["geoNetwork.country__google_at_asia"] + train["geoNetwork.country__google_at_middle_east"] 
test["geoNetwork.country__has_google"] = test["geoNetwork.country__google_at_north_america"]+ test["geoNetwork.country__google_at_latin_america"]  + test["geoNetwork.country__google_at_europe"] +test["geoNetwork.country__google_at_asia"] + test["geoNetwork.country__google_at_middle_east"] 



Working on size for  geoNetwork.country
Working on spending for  geoNetwork.country
Making Features size features for  geoNetwork.country
Making Features spending features for  geoNetwork.country


##  geoNetwork.metro & subContinent

We will let categorical Feature selection handle these feature

##  geoNetwork.region

In [16]:
# Make geo by size feature
geo_by_size_spenders("geoNetwork.region")

Working on size for  geoNetwork.region
Working on spending for  geoNetwork.region
Making Features size features for  geoNetwork.region
Making Features spending features for  geoNetwork.region


In [17]:
print(f" Train shape is {train.shape} and test shape is {test.shape}")
#train.columns

 Train shape is (903653, 71) and test shape is (804684, 71)


Index(['channelGrouping', 'fullVisitorId', 'sessionId', 'visitId',
       'visitNumber', 'device.browser', 'device.deviceCategory',
       'device.isMobile', 'device.operatingSystem', 'geoNetwork.city',
       'geoNetwork.continent', 'geoNetwork.country', 'geoNetwork.metro',
       'geoNetwork.networkDomain', 'geoNetwork.region',
       'geoNetwork.subContinent', 'totals.bounces', 'totals.hits',
       'totals.newVisits', 'totals.pageviews', 'totals.transactionRevenue',
       'trafficSource.adContent', 'trafficSource.campaign',
       'trafficSource.isTrueDirect', 'trafficSource.keyword',
       'trafficSource.medium', 'trafficSource.referralPath',
       'trafficSource.source', 'sess_date_dow', 'sess_date_hours',
       'sess_date_dom', 'device.operatingSystem__Bucket',
       'device.operatingSystem__Linux', 'device.operatingSystem__Macintosh',
       'device.operatingSystem__iOS', 'device.operatingSystem__Windows',
       'device.operatingSystem__Android', 'device.operatingSystem__

# Automatic Feature Engineering 

In [18]:
# Remove Bad Columns 
cols_to_drop = ["geoNetwork.networkDomain","trafficSource.keyword","trafficSource.referralPath"]
for each in [train,test]:
    each.drop(cols_to_drop,axis=1,inplace=True)

In [19]:
from helper.CategoricalLevelSelector import CategoricalLevelSelection
cls = CategoricalLevelSelection() 
cls.fit(train,key="fullVisitorId",cols_to_ignore=["sessionId","device.operatingSystem__Bucket"],verbose=True)
cols = ['channelGrouping', 'fullVisitorId', 'sessionId', 'visitId',
       'visitNumber','device.browser',
       'device.deviceCategory', 'device.operatingSystem', 'geoNetwork.city',
       ]
train = cls.transform(train,verbose=True)
test = cls.transform(test,verbose=True)

Working on channelGrouping
Working on device.browser
Working on device.deviceCategory
Working on device.operatingSystem
Working on device.operatingSystem__Bucket
Working on geoNetwork.city
Working on geoNetwork.continent
Working on geoNetwork.country
Working on geoNetwork.metro
Working on geoNetwork.region
Working on geoNetwork.subContinent
Working on trafficSource.adContent
Working on trafficSource.campaign
Working on trafficSource.medium
Working on trafficSource.source
changing channelGrouping
changing device.browser
changing device.deviceCategory
changing device.operatingSystem
changing geoNetwork.city
changing geoNetwork.continent
changing geoNetwork.country
changing geoNetwork.metro
changing geoNetwork.region
changing geoNetwork.subContinent
changing trafficSource.adContent


/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:468: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask &= (ar1 != a)


changing trafficSource.campaign
changing trafficSource.medium
changing trafficSource.source
changing device.operatingSystem__Bucket
changing channelGrouping
changing device.browser
changing device.deviceCategory
changing device.operatingSystem
changing geoNetwork.city
changing geoNetwork.continent
changing geoNetwork.country
changing geoNetwork.metro
changing geoNetwork.region
changing geoNetwork.subContinent
changing trafficSource.adContent
changing trafficSource.campaign
changing trafficSource.medium
changing trafficSource.source
changing device.operatingSystem__Bucket


In [20]:

excluded_features = ['date', 'fullVisitorId', 'sessionId', 
                     'totals.transactionRevenue', 
                     'visitId', 'visitStartTime']

categorical_features = [
    _f for _f in train.columns
    if (_f not in excluded_features) & (train[_f].dtype == 'O')]


In [21]:
for f in categorical_features:
    train[f], indexer = pd.factorize(train[f])
    test[f] = indexer.get_indexer(test[f])

(804684, 68)

In [23]:
def get_folds(df=None, n_splits=5):
    """Returns dataframe indices corresponding to Visitors Group KFold from Oliver's Code"""
    # Get sorted unique visitors
    unique_vis = np.array(sorted(df['fullVisitorId'].unique()))

    # Get folds
    folds = GroupKFold(n_splits=n_splits)
    fold_ids = []
    ids = np.arange(df.shape[0])
    for trn_vis, val_vis in folds.split(X=unique_vis, y=unique_vis, groups=unique_vis):
        fold_ids.append(
            [
                ids[df['fullVisitorId'].isin(unique_vis[trn_vis])],
                ids[df['fullVisitorId'].isin(unique_vis[val_vis])]
            ]
        )

    return fold_ids